In [25]:
import pandas as pd
import numpy as np
import operator

In [26]:
submission_file_list = ['submissions/MLPR_image_1000_des_BOW_all_23.csv',
                        'submissions/PLSR_100_BOW_all_pca_2048_tags_80_cos_0.26727.csv',
                        'submissions/0.352_plsr_800_pool5_to_bow_all_pca_2048.csv',
                        'submissions/reverse_count_weight_canary_word_count_with_tag_2048_no_caption_0.40498.csv']

In [27]:
submission_weight = [0.23, 0.26727, 0.352, 0.40498]

output_name = 'submissions/ensembled_0.40498_0.352_0.26727_0.23_models_weight.csv'

In [28]:
def calc_single(pred_list):
    score_dict = {}
    for i in range(len(pred_list)):
        for j in range(len(pred_list[i])):
            if pred_list[i][j] not in score_dict:
                score_dict[pred_list[i][j]] = 0.0
            score_dict[pred_list[i][j]] += float(len(pred_list[i]) - j) / len(pred_list[i]) * submission_weight[i]
    sorted_score = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
    return map(lambda x: x[0], sorted_score[:20])

In [29]:
def ensemble():
    res = []
    dfs = [pd.read_csv(file_path, encoding='utf-8') for file_path in submission_file_list]
    for i in range(dfs[0].shape[0]):
        pred_list = [df.iloc[i]['Top_20_Image_IDs'].split(' ') for df in dfs]
        res.append(' '.join(calc_single(pred_list)))
    print len(res)
    return pd.DataFrame({'Descritpion_ID': dfs[0]['Descritpion_ID'].tolist(), 'Top_20_Image_IDs': res})

In [30]:
df_ensembled = ensemble()

2000


In [31]:
print df_ensembled

     Descritpion_ID                                   Top_20_Image_IDs
0             0.txt  1105.jpg 1480.jpg 338.jpg 105.jpg 1344.jpg 170...
1             1.txt  833.jpg 818.jpg 457.jpg 589.jpg 80.jpg 314.jpg...
2             2.txt  838.jpg 1724.jpg 159.jpg 1471.jpg 1840.jpg 118...
3             3.txt  1514.jpg 26.jpg 1024.jpg 1215.jpg 1765.jpg 177...
4             4.txt  742.jpg 1231.jpg 922.jpg 594.jpg 292.jpg 1269....
5             5.txt  771.jpg 814.jpg 51.jpg 492.jpg 1335.jpg 1693.j...
6             6.txt  1855.jpg 1216.jpg 1815.jpg 1660.jpg 1701.jpg 1...
7             7.txt  689.jpg 448.jpg 627.jpg 168.jpg 409.jpg 1285.j...
8             8.txt  419.jpg 719.jpg 531.jpg 103.jpg 1283.jpg 1847....
9             9.txt  614.jpg 549.jpg 770.jpg 1886.jpg 899.jpg 1795....
10           10.txt  634.jpg 1840.jpg 1366.jpg 42.jpg 696.jpg 1421....
11           11.txt  814.jpg 492.jpg 771.jpg 1693.jpg 51.jpg 846.jp...
12           12.txt  1653.jpg 837.jpg 1950.jpg 1656.jpg 1031.jpg 67...
13    

In [32]:
df_ensembled.to_csv(output_name, index=False)